<a href="https://colab.research.google.com/github/OctavioGuaschi/IA901-2023S1/blob/main/projetos/Human_physical_position_detection/notebooks/Splitdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
data_dir = "gdrive/MyDrive/Projeto/data/raw/Human_silhouettes_original_dataset"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# Imports

# import the needed libs

from __future__ import print_function, division
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torchsummary import summary
import matplotlib.pyplot as plt
import time
import os
import copy
from google.colab import files

plt.ion()   # interactive mode

In [ ]:
class ImageFolderWithPath(datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return img, target, path

In [ ]:
image_datasets = ImageFolderWithPath(os.path.join(data_dir))
# Train (60%), Val (20%), Test (20%)
train_size = int(0.6 * len(image_datasets))
test_size = int(0.2 * len(image_datasets))
val_size = len(image_datasets) - train_size - test_size
train_dataset_1, val_dataset_1, test_dataset_1 = torch.utils.data.random_split(image_datasets, [train_size, val_size, test_size])



In [ ]:
train_size

2169

In [ ]:
print(image_datasets.imgs[0])

('gdrive/MyDrive/Projeto/data/raw/Human_silhouettes_original_dataset/tmp/lying/1.jpg', 0)


In [ ]:
print(train_dataset_1.indices)

[909, 123, 133, 3000, 933, 1264, 2961, 1732, 1197, 50, 2009, 1900, 2722, 2275, 508, 3581, 1730, 529, 452, 2111, 591, 637, 3399, 2154, 1978, 3351, 2392, 3397, 2672, 1150, 2580, 1847, 3114, 1308, 1884, 2999, 535, 2891, 3408, 1389, 2981, 427, 766, 908, 3475, 105, 2803, 1300, 435, 1105, 511, 2487, 66, 3495, 2784, 1606, 2277, 378, 1241, 1116, 1520, 1179, 1083, 1775, 2890, 1939, 1799, 2015, 1016, 1937, 2383, 2746, 1409, 2354, 1260, 588, 2359, 1966, 1480, 1092, 662, 1762, 1575, 2254, 269, 410, 847, 2883, 3349, 1478, 2335, 2531, 2250, 2549, 1710, 3199, 2920, 30, 2555, 1529, 2295, 3551, 1153, 3159, 1743, 1337, 2299, 1191, 1494, 1123, 967, 2132, 3545, 1771, 1794, 1008, 2350, 2787, 364, 2113, 0, 2972, 2263, 126, 187, 2765, 2028, 3276, 3413, 2895, 2495, 2186, 1274, 688, 1816, 2711, 583, 97, 1485, 2385, 676, 1431, 2315, 1215, 2521, 2024, 417, 2199, 3324, 3068, 2563, 1464, 1354, 927, 3308, 3182, 3271, 1121, 202, 1074, 499, 1242, 3458, 1790, 3262, 2051, 1696, 1645, 2273, 1036, 398, 1760, 1662, 2481, 

In [ ]:
train_dataset = train_dataset_1.indices

In [ ]:
print(val_dataset_1.indices)

[1304, 3212, 3318, 805, 463, 280, 1687, 284, 585, 1943, 2293, 248, 3206, 914, 1537, 1953, 2743, 3547, 1014, 1890, 816, 2729, 1091, 3044, 1345, 1332, 507, 3507, 3048, 2793, 3558, 431, 574, 3332, 519, 2283, 1272, 1062, 1582, 2067, 3150, 754, 996, 503, 2560, 196, 312, 516, 1408, 2218, 1315, 2127, 802, 3556, 3406, 3509, 429, 1526, 1617, 672, 708, 3336, 2731, 1401, 648, 68, 3403, 400, 8, 2511, 1922, 3516, 1033, 694, 3452, 2031, 136, 1556, 444, 797, 2736, 726, 64, 175, 1061, 899, 2458, 3034, 3204, 1212, 1017, 470, 1026, 1075, 416, 630, 3343, 3404, 1444, 2761, 2013, 506, 1076, 869, 3065, 2971, 2270, 1339, 3607, 3166, 860, 3272, 173, 3211, 2985, 3274, 582, 2578, 1227, 1519, 1653, 3051, 162, 3111, 1566, 90, 664, 2488, 1094, 2011, 731, 3191, 204, 1741, 1833, 1135, 2880, 1240, 3263, 3510, 2937, 806, 502, 274, 344, 413, 1536, 3165, 1030, 1009, 2424, 3131, 2584, 997, 1257, 1712, 3462, 3388, 1239, 2442, 437, 2750, 711, 1273, 2504, 2248, 1636, 1827, 820, 2444, 434, 1923, 276, 3384, 1078, 3444, 2910, 

In [ ]:
val_dataset = val_dataset_1.indices

In [ ]:
print(test_dataset_1.indices)

[2879, 1702, 3148, 1038, 325, 438, 2376, 3205, 1234, 1490, 3569, 3578, 3358, 2867, 2756, 848, 1363, 2297, 240, 343, 2126, 1496, 3451, 1353, 99, 3518, 1543, 912, 481, 2966, 2439, 1894, 52, 744, 1651, 1114, 2789, 377, 878, 2369, 840, 141, 921, 122, 522, 116, 2211, 2556, 2990, 298, 1330, 474, 46, 514, 1102, 1612, 3528, 3003, 3553, 2259, 3095, 900, 685, 483, 1204, 1599, 2644, 1365, 3301, 1573, 3417, 1229, 2648, 566, 1512, 1744, 1711, 1306, 1050, 2715, 783, 1949, 2427, 3519, 2720, 3501, 1896, 547, 3549, 1461, 56, 2373, 1254, 1608, 1716, 910, 487, 3047, 807, 2406, 2894, 1414, 2318, 2191, 1675, 2089, 1866, 2450, 2055, 1292, 3186, 2328, 2730, 2620, 610, 3541, 712, 1763, 2179, 403, 2367, 81, 3441, 2693, 209, 1103, 973, 835, 863, 195, 974, 10, 1920, 1096, 385, 2205, 1400, 19, 2662, 1053, 2930, 1545, 2608, 1178, 3530, 2285, 1764, 3573, 200, 2196, 167, 3259, 1107, 139, 1499, 3387, 1384, 2956, 1705, 785, 2243, 3197, 3300, 2264, 1770, 2643, 164, 851, 1593, 182, 80, 1870, 1476, 451, 1230, 1774, 1908,

In [ ]:
test_dataset = test_dataset_1.indices

In [ ]:
for i in range(0, train_size):
  print( str(i) + " " + image_datasets.imgs[train_dataset[i]][0].split("gdrive/MyDrive/Projeto/data/raw/Human_silhouettes_original_dataset/tmp")[1])
  img = Image.open(image_datasets.imgs[train_dataset[i]][0])
  img.save(('gdrive/MyDrive/Projeto/data/raw/human_silhouettes_split_no_bending/train' + image_datasets.imgs[train_dataset[i]][0].split("gdrive/MyDrive/Projeto/data/raw/Human_silhouettes_original_dataset/tmp")[1]))

0 /lying/722.jpg
1 /lying/1109.jpg
2 /lying/1118.jpg
3 /standing/444.jpg
4 /lying/744.jpg
5 /sitting/1041.jpg
6 /standing/409.jpg
7 /sitting/383.jpg
8 /lying/982.jpg
9 /lying/1043.jpg
10 /sitting/632.jpg
11 /sitting/534.jpg
12 /standing/194.jpg
13 /sitting/872.jpg
14 /lying/376.jpg
15 /standing/968.jpg
16 /sitting/381.jpg
17 /lying/395.jpg
18 /lying/325.jpg
19 /sitting/724.jpg
20 /lying/443.jpg
21 /lying/485.jpg
22 /standing/803.jpg
23 /sitting/763.jpg
24 /sitting/604.jpg
25 /standing/760.jpg
26 /sitting/978.jpg
27 /standing/801.jpg
28 /standing/149.jpg
29 /lying/94.jpg
30 /standing/1146.jpg
31 /sitting/487.jpg
32 /standing/547.jpg
33 /sitting/1081.jpg
34 /sitting/52.jpg
35 /standing/443.jpg
36 /lying/40.jpg
37 /standing/346.jpg
38 /standing/811.jpg
39 /sitting/1154.jpg
40 /standing/427.jpg
41 /lying/302.jpg
42 /lying/594.jpg
43 /lying/721.jpg
44 /standing/872.jpg
45 /lying/1093.jpg
46 /standing/267.jpg
47 /sitting/1074.jpg
48 /lying/31.jpg
49 /lying/9.jpg
50 /lying/379.jpg
51 /standin

In [ ]:
train_dataset[i]

(<PIL.Image.Image image mode=RGB size=512x512 at 0x7FB0A5EC5750>,
 1,
 'gdrive/MyDrive/Projeto/data/raw/human_silhouettes/sitting/1057.jpg')

In [ ]:
for i in range(0, test_size):
  print( str(i) + " " + image_datasets.imgs[test_dataset[i]][0].split("gdrive/MyDrive/Projeto/data/raw/Human_silhouettes_original_dataset/tmp")[1])
  img = Image.open(image_datasets.imgs[test_dataset[i]][0])
  img.save(('gdrive/MyDrive/Projeto/data/raw/human_silhouettes_split_no_bending/test/' + image_datasets.imgs[test_dataset[i]][0].split("gdrive/MyDrive/Projeto/data/raw/Human_silhouettes_original_dataset/tmp")[1]))

0 /standing/335.jpg
1 /sitting/356.jpg
2 /standing/578.jpg
3 /lying/839.jpg
4 /lying/210.jpg
5 /lying/312.jpg
6 /sitting/963.jpg
7 /standing/629.jpg
8 /sitting/1014.jpg
9 /sitting/165.jpg
10 /standing/957.jpg
11 /standing/965.jpg
12 /standing/767.jpg
13 /standing/324.jpg
14 /standing/224.jpg
15 /lying/668.jpg
16 /sitting/1130.jpg
17 /sitting/892.jpg
18 /lying/134.jpg
19 /lying/227.jpg
20 /sitting/738.jpg
21 /sitting/170.jpg
22 /standing/850.jpg
23 /sitting/1121.jpg
24 /lying/1088.jpg
25 /standing/910.jpg
26 /sitting/212.jpg
27 /lying/725.jpg
28 /lying/351.jpg
29 /standing/413.jpg
30 /standing/1019.jpg
31 /sitting/529.jpg
32 /lying/1045.jpg
33 /lying/574.jpg
34 /sitting/31.jpg
35 /lying/907.jpg
36 /standing/254.jpg
37 /lying/258.jpg
38 /lying/695.jpg
39 /sitting/957.jpg
40 /lying/660.jpg
41 /lying/1125.jpg
42 /lying/733.jpg
43 /lying/1108.jpg
44 /lying/389.jpg
45 /lying/1102.jpg
46 /sitting/814.jpg
47 /standing/1124.jpg
48 /standing/435.jpg
49 /lying/187.jpg
50 /sitting/1100.jpg
51 /lyi

In [ ]:
for i in range(0,val_size):
  print( str(i) + " " + image_datasets.imgs[val_dataset[i]][0].split("gdrive/MyDrive/Projeto/data/raw/Human_silhouettes_original_dataset/tmp")[1])
  img = Image.open(image_datasets.imgs[val_dataset[i]][0])
  img.save(('gdrive/MyDrive/Projeto/data/raw/human_silhouettes_split_no_bending/val/' + image_datasets.imgs[val_dataset[i]][0].split("gdrive/MyDrive/Projeto/data/raw/Human_silhouettes_original_dataset/tmp")[1]))

0 /sitting/1078.jpg
1 /standing/635.jpg
2 /standing/730.jpg
3 /lying/629.jpg
4 /lying/335.jpg
5 /lying/170.jpg
6 /sitting/342.jpg
7 /lying/174.jpg
8 /lying/438.jpg
9 /sitting/573.jpg
10 /sitting/889.jpg
11 /lying/141.jpg
12 /standing/63.jpg
13 /lying/727.jpg
14 /sitting/207.jpg
15 /sitting/582.jpg
16 /standing/212.jpg
17 /standing/937.jpg
18 /lying/817.jpg
19 /sitting/525.jpg
20 /lying/639.jpg
21 /standing/20.jpg
22 /lying/887.jpg
23 /standing/484.jpg
24 /sitting/1114.jpg
25 /sitting/1102.jpg
26 /lying/375.jpg
27 /standing/900.jpg
28 /standing/488.jpg
29 /standing/258.jpg
30 /standing/947.jpg
31 /lying/306.jpg
32 /lying/428.jpg
33 /standing/743.jpg
34 /lying/386.jpg
35 /sitting/88.jpg
36 /sitting/1049.jpg
37 /lying/860.jpg
38 /sitting/248.jpg
39 /sitting/685.jpg
40 /standing/58.jpg
41 /lying/583.jpg
42 /lying/800.jpg
43 /lying/371.jpg
44 /standing/1128.jpg
45 /lying/1175.jpg
46 /lying/2.jpg
47 /lying/383.jpg
48 /sitting/1171.jpg
49 /sitting/820.jpg
50 /sitting/1088.jpg
51 /sitting/739.

In [ ]:
print(train_dataset.indices[:])

AttributeError: ignored

In [ ]:
print(test_dataset.indices[:])

AttributeError: ignored

In [ ]:
print(val_dataset.indices[:])

AttributeError: ignored

17 de maio de 2023 as 17h25